In [ ]:
pip install git+https://github.com/bianchimario/MARS

In [1]:
from MARS import MARS
from downtime import load_dataset
import lightgbm as lgb
from sklearn.metrics import classification_report
from datatime.conversion import awkward_to_flat
import awkward as ak

In [2]:
# Importing dataset

d = load_dataset('Cricket')
X_train = d.X_train
X_test = d.X_test
y_train = d.y_train
y_test = d.y_test
print(d)

X_train: (108, 6, 1197)
X_test: (72, 6, 1197)
y_train: (108,)
y_test: (72,)
Metadata:
{'labels': {'0': '1.0',
            '1': '10.0',
            '10': '8.0',
            '11': '9.0',
            '2': '11.0',
            '3': '12.0',
            '4': '2.0',
            '5': '3.0',
            '6': '4.0',
            '7': '5.0',
            '8': '6.0',
            '9': '7.0'},
 'name': 'Cricket',
 'task': 'classification'}


# MARS

In [8]:
# Extracting shapelets
mars = MARS(num_shapelets=100, 
            max_len=50, 
            min_len=10, 
            seed=42, 
            shapelet_indexes=True, # to save the index of the time series from which the shapelet is extracted
            indexes=True, # to save the index of the best match between shaplet and time series
            n_jobs=-1 
            )

In [9]:
# Fitting MARS
mars.fit(X_train)

MARS(indexes=True, max_len=50, min_len=10, num_shapelets=100, seed=42,
     shapelet_indexes=[81, 14, 3, 94, 35, 31, 28, 17, 94, 13, 86, 94, 69, 11,
                       75, 54, 4, 3, 11, 27, 29, 64, 77, 3, 71, 25, 91, 83, 89,
                       69, ...])

In [10]:
# Calculating distances (i.e. Shapelet Transform)
X_train_transformed, train_idxs = mars.transform(X_train)
X_test_transformed, test_idxs = mars.transform(X_test)

# If indexes=False:
# X_train_transformed, train_idxs = mars.transform(X_train)
# X_test_transformed, test_idxs = mars.transform(X_test)

# Classification

In [12]:
# Applying classifier of choice
clf = lgb.LGBMClassifier()
clf.fit(X_train_transformed, y_train)

LGBMClassifier()

In [13]:
# Generating predictions
y_pred_test = clf.predict(X_test_transformed)

print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.83      0.83      0.83         6
           1       1.00      1.00      1.00         6
           2       0.86      1.00      0.92         6
           3       1.00      0.50      0.67         6
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00         6
           7       1.00      1.00      1.00         6
           8       0.86      1.00      0.92         6
           9       1.00      1.00      1.00         6
          10       1.00      1.00      1.00         6
          11       0.86      1.00      0.92         6

    accuracy                           0.94        72
   macro avg       0.95      0.94      0.94        72
weighted avg       0.95      0.94      0.94        72

